In [66]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.decomposition import PCA
from xgboost import XGBClassifier
import joblib
import pickle

In [67]:
clf = joblib.load("./generalization_model/training_classifier.pkl")
clf_scaler = joblib.load("./generalization_model/training_scaler.pkl")
clf_pca = joblib.load("./generalization_model/training_pca.pkl")


In [68]:
df_test = pd.read_csv('./data/test_data.csv')
Index = df_test['Index']
df_test= df_test.drop(columns=['Index'])

In [69]:
X_test = df_test
X_test_np = clf_scaler.transform(X_test)
X_test_pca = clf_pca.transform(X_test_np)
labels=clf.predict(X_test_pca)
X_test['cluster']=labels
print(X_test['cluster'].value_counts())

df_predict = pd.DataFrame({
    'Index':Index,
    'cluster':labels
})
df_predict['Bankrupt?'] = -1
# if label =2, predict 0
df_predict.loc[df_predict['cluster'] == 2, 'Bankrupt?'] = 0

cluster
3    353
1    268
4    243
0     91
2     57
Name: count, dtype: int64


In [70]:
#extract the label 0
idx_0 = df_predict['cluster'] == 0
X_test_subgroup0 = X_test.loc[idx_0, :]
subgroup0_extract_features = joblib.load("./subgroup0/extract_features.pkl")
subgroup0_scaler = joblib.load("./subgroup0/scaler.pkl")
subgroup0_model = joblib.load("./subgroup0/best_stack_model.pkl")
X_test_subgroup0_selected =X_test_subgroup0[subgroup0_extract_features]
X_test_subgroup0_scaled = subgroup0_scaler.transform(X_test_subgroup0_selected)
preds_0 = subgroup0_model.predict(X_test_subgroup0_scaled)
df_predict.loc[idx_0, 'Bankrupt?'] = preds_0
df_predict['Bankrupt?'].value_counts()

Bankrupt?
-1    864
 0    126
 1     22
Name: count, dtype: int64

In [71]:
#extract the label 1
idx_1 = df_predict['cluster'] == 1

X_test_subgroup1 = X_test.loc[idx_1, :]
X_test_subgroup1 = X_test_subgroup1.drop(columns=['cluster',' Net Income Flag'])
X_test_subgroup1 = X_test_subgroup1.values

subgroup1_pca = joblib.load("./subgroup1/XPCA.pkl")
subgroup1_scaler = joblib.load("./subgroup1/StScaler.pkl")
subgroup1_model = joblib.load("./subgroup1/StackCLF.pkl")

X_test_subgroup1_scaled = subgroup1_scaler.transform(X_test_subgroup1)
X_test_subgroup1_selected =subgroup1_pca.transform(X_test_subgroup1_scaled)

preds_1 = subgroup1_model.predict(X_test_subgroup1_selected)
df_predict.loc[idx_1, 'Bankrupt?'] = preds_1
df_predict['Bankrupt?'].value_counts()

Bankrupt?
-1    596
 0    394
 1     22
Name: count, dtype: int64

In [72]:
#extract the label 3
idx_3 = df_predict['cluster'] == 3

X_test_subgroup3 = X_test.loc[idx_3, :]
subgroup3_extract_features = joblib.load("./subgroup3/feature_names.pkl")
subgroup3_scaler = joblib.load("./subgroup3/scaler.pkl")
subgroup3_model = joblib.load("./subgroup3/stacking_model.pkl")
X_test_subgroup3_selected =X_test_subgroup3[subgroup3_extract_features]
X_test_subgroup3_scaled = subgroup3_scaler.transform(X_test_subgroup3_selected)
preds_3 = subgroup3_model.predict(X_test_subgroup3_scaled)
df_predict.loc[idx_3, 'Bankrupt?'] = preds_3
df_predict['Bankrupt?'].value_counts()

Bankrupt?
 0    738
-1    243
 1     31
Name: count, dtype: int64

In [73]:
#extract the label 4
idx_4 = df_predict['cluster'] == 4

subgroup4_extract = joblib.load("./subgroup4/extract_features.pkl")
subgroup4_pca = joblib.load("./subgroup4/pca.pkl")
subgroup4_scaler = joblib.load("./subgroup4/scaler.pkl")
subgroup4_model = joblib.load("./subgroup4/best_stack_model.pkl")

X_test_subgroup4 = X_test.loc[idx_4, subgroup4_extract]
X_test_subgroup4_scaled = subgroup4_scaler.transform(X_test_subgroup4)
X_test_subgroup4_selected =subgroup4_pca.transform(X_test_subgroup4_scaled)

preds_4 = subgroup4_model.predict(X_test_subgroup4_selected)
df_predict.loc[idx_4, 'Bankrupt?'] = preds_4
df_predict['Bankrupt?'].value_counts()

Bankrupt?
0    964
1     48
Name: count, dtype: int64

In [74]:
print(df_predict)
df_predict = df_predict.drop(columns=['cluster'])
df_predict.to_csv('./Group1_Generalization.csv')

      Index  cluster  Bankrupt?
0         0        1          0
1         1        3          0
2         2        1          0
3         3        4          0
4         4        0          0
...     ...      ...        ...
1007   1007        4          0
1008   1008        3          0
1009   1009        3          0
1010   1010        1          0
1011   1011        3          0

[1012 rows x 3 columns]
